In [7]:
from arcgis.gis import GIS
from arcgis.graph import KnowledgeGraph
from uuid import UUID
from pprint import pprint

### Connect

In [8]:
username = "analyst"
password = "Esri.4.GIS"
url = "https://ebase.ad.local/server/rest/services/Hosted/SupplyChain/KnowledgeGraphServer"
gis = GIS(r"https://ebase.ad.local/portal", username, password)
knowledge_graph = KnowledgeGraph(url=url, gis=gis)
knowledge_graph

### Get list of distributor names to process

In [9]:
distributorNameQuery = """MATCH (p:ProcessingPlant)
WITH DISTINCT p.Distributors as d
RETURN d"""
x=knowledge_graph.query(distributorNameQuery)
y = []
distributorNames = []
for i in range(len(x)):
    y.extend(x[i])
for item in y:
    temp = item.split(", ")
    for s in temp:
        distributorNames.append(s)
distributorNames = list(set(distributorNames))
print(distributorNames)

['Distributor 18', 'Distributor 12', 'Distributor 5', 'Distributor 13', 'Distributor 2', 'Distributor 1', 'Distributor 20', 'Distributor 16', 'Distributor 6', 'Distributor 4', 'Distributor 23', 'Distributor 21', 'Distributor 11', 'Distributor 17', 'Distributor 22', 'Distributor 8', 'Distributor 19', 'Distributor 3', 'Distributor 7', 'Distributor 14', 'Distributor 10', 'Distributor 15', 'Distributor 9']


### Query

In [10]:
def submitQuery(distributorName):
    opQuery = """MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "{0}")
    MATCH (d:Distributor)
    WHERE d.DistributorName = "{0}"
    RETURN p, d""".format(distributorName)
    print(opQuery)

    queryResults = knowledge_graph.query(opQuery)
    return queryResults

### Given returned nodes, create relationships

In [11]:
def createRelationships(queryResults):
    countRels = 0
    for p, d in queryResults:
        print("Related nodes:")
        print(p['_properties']['ProcessingPlantName'])
        #print(p['_id'])
        #print(p['_properties']['Material'])
        print(d['_properties']['DistributorName'])
        print(d['_id'])
        #print(d['_properties']['Material'])
        #print()
        add_relationship = {
            "_objectType": "relationship",
            "_typeName": "SellsTo",
            "_originEntityId": UUID(f"{d['_id']}"), # Distributor ID
            "_destinationEntityId": UUID(f"{p['_id']}") # Plant ID
        }
        knowledge_graph.apply_edits(adds=[add_relationship])
        countRels += 1
        #print("EDITS MADE")
    print(f"All done! {countRels} relationships created.")

### Perform the edits

In [12]:
for distributor in distributorNames:
    print(f"Finding related Processing Plants for {distributor}")
    queryNodes = submitQuery(distributor)
    createRelationships(queryNodes) 
print("End of script.")

Finding related Processing Plants for Distributor 18
MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "Distributor 18")
    MATCH (d:Distributor)
    WHERE d.DistributorName = "Distributor 18"
    RETURN p, d
Related nodes:
Plant 4
Distributor 18
07a38bb5-9717-480c-8c55-a0dea774a0b4
Related nodes:
Plant 7
Distributor 18
07a38bb5-9717-480c-8c55-a0dea774a0b4
All done! 2 relationships created.
Finding related Processing Plants for Distributor 12
MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "Distributor 12")
    MATCH (d:Distributor)
    WHERE d.DistributorName = "Distributor 12"
    RETURN p, d
Related nodes:
Plant 6
Distributor 12
38aed19b-f382-443f-8f43-0b94ec4a072b
All done! 1 relationships created.
Finding related Processing Plants for Distributor 5
MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "Distributor 5")
    MATCH (d:Distributor)
    WHERE d.Distributor